In [1]:
import json
from openai import OpenAI
from memory_manager import create_memory, retrieve_memory, delete_memory

In [11]:
client = OpenAI(api_key="your_api_key")

In [3]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "create_memory",
            "description": "Uses this function to store a new piece of information or fact. For example, when the user says 'Remember that my name is Prudhvi'.",
            "parameters": {
                "type": "object",
                "properties": {
                    "content": {
                        "type": "string",
                        "description": "The specific fact or piece of information to remembered.",
                    },
                },
                "required": ["content"]
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "retrieve_memory",
            "description": "Uses this function to find a relevent memory when the user asks a question. For example, when the user asks 'What is my name?'",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The user's question or the topic to search for in memory.",
                    },
                    "n_results": {
                        "type": "integer",
                        "default": 1,
                        "description": "The number of results to return."
                    },
                },
                "required": ["query"]
            },
        },
    },
    {
        "type": "function",
        "function": {
            "name": "delete_memory",
            "description": "Uses this function to delete a specific memory when the user explicitly says to forget something. For example, when the user says 'Forget that my name is Prudhvi'.",
            "parameters": {
                "type": "object",
                "properties": {
                    "content_to_delete": {
                        "type": "string",
                        "description": "The specific content of the memory to deleted.",
                    },
                },
                "required": ["content_to_delete"]
            },
        },
    }
]

In [4]:
available_functions = {
    "create_memory": create_memory,
    "retrieve_memory": retrieve_memory,
    "delete_memory": delete_memory,
}

In [5]:
 messages = [
        {
            "role": "system",
            "content": "You are a helpful AI assistant with access to tools for long-term memory."
        }
    ]

In [6]:
user_input = input("You: ")

You:  my name is prudhvi.


In [7]:
messages.append({
            "role": "user",
            "content": user_input
        })

In [8]:
messages

[{'role': 'system',
  'content': 'You are a helpful AI assistant with access to tools for long-term memory.'},
 {'role': 'user', 'content': 'my name is prudhvi.'}]

In [14]:
response = client.chat.completions.create(
            model = "gpt-4o-mini",
            messages = messages,
            tools = tools,
            tool_choice = "auto",
        )

In [15]:
response

ChatCompletion(id='chatcmpl-C1E6LhS8I8MQAgGPlyDMIqhcEfsOL', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Ks13T3atDv9O78id9SutnYns', function=Function(arguments='{"content":"User\'s name is Prudhvi."}', name='create_memory'), type='function')]))], created=1754408569, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=21, prompt_tokens=235, total_tokens=256, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [16]:
response.choices[0]

Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Ks13T3atDv9O78id9SutnYns', function=Function(arguments='{"content":"User\'s name is Prudhvi."}', name='create_memory'), type='function')]))

In [17]:
response_message =  response.choices[0].message

In [18]:
response_message

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Ks13T3atDv9O78id9SutnYns', function=Function(arguments='{"content":"User\'s name is Prudhvi."}', name='create_memory'), type='function')])

In [19]:
messages.append(response_message)

In [20]:
messages

[{'role': 'system',
  'content': 'You are a helpful AI assistant with access to tools for long-term memory.'},
 {'role': 'user', 'content': 'my name is prudhvi.'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Ks13T3atDv9O78id9SutnYns', function=Function(arguments='{"content":"User\'s name is Prudhvi."}', name='create_memory'), type='function')])]

In [21]:
tool_calls = response_message.tool_calls

In [22]:
tool_calls

[ChatCompletionMessageToolCall(id='call_Ks13T3atDv9O78id9SutnYns', function=Function(arguments='{"content":"User\'s name is Prudhvi."}', name='create_memory'), type='function')]

In [23]:
tool_calls[0].function

Function(arguments='{"content":"User\'s name is Prudhvi."}', name='create_memory')

In [24]:
function_name = tool_calls[0].function.name

In [25]:
function_name

'create_memory'

In [26]:
function_to_call = available_functions[function_name]

In [27]:
function_to_call

<function memory_manager.create_memory(content: str)>

In [28]:
tool_calls[0].function.arguments

'{"content":"User\'s name is Prudhvi."}'

In [29]:
function_args = json.loads(tool_calls[0].function.arguments)

In [30]:
function_args

{'content': "User's name is Prudhvi."}

In [31]:
function_response = function_to_call(**function_args)

MEMORY CREATED: 'User's name is Prudhvi.' with ID 63d19247-7b2d-4a70-b45b-36b21a65bbf3


In [32]:
tool_calls[0].id

'call_Ks13T3atDv9O78id9SutnYns'

In [33]:
messages.append(
    {
        "tool_call_id" : tool_calls[0].id,
        "role" : "tool",
        "name" : function_name,
        "content" : function_response,
    }
)

In [34]:
messages

[{'role': 'system',
  'content': 'You are a helpful AI assistant with access to tools for long-term memory.'},
 {'role': 'user', 'content': 'my name is prudhvi.'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Ks13T3atDv9O78id9SutnYns', function=Function(arguments='{"content":"User\'s name is Prudhvi."}', name='create_memory'), type='function')]),
 {'tool_call_id': 'call_Ks13T3atDv9O78id9SutnYns',
  'role': 'tool',
  'name': 'create_memory',
  'content': "OK, I will remember that: 'User's name is Prudhvi.'"}]

In [35]:
second_response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages = messages,
)

In [36]:
second_response

ChatCompletion(id='chatcmpl-C1E6mcMlxr84tNLApax8zjkHqkr5L', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Nice to meet you, Prudhvi! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1754408596, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=16, prompt_tokens=78, total_tokens=94, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [37]:
final_response = second_response.choices[0].message.content

In [38]:
final_response

'Nice to meet you, Prudhvi! How can I assist you today?'

In [39]:
messages.append(second_response.choices[0].message)

In [40]:
messages

[{'role': 'system',
  'content': 'You are a helpful AI assistant with access to tools for long-term memory.'},
 {'role': 'user', 'content': 'my name is prudhvi.'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Ks13T3atDv9O78id9SutnYns', function=Function(arguments='{"content":"User\'s name is Prudhvi."}', name='create_memory'), type='function')]),
 {'tool_call_id': 'call_Ks13T3atDv9O78id9SutnYns',
  'role': 'tool',
  'name': 'create_memory',
  'content': "OK, I will remember that: 'User's name is Prudhvi.'"},
 ChatCompletionMessage(content='Nice to meet you, Prudhvi! How can I assist you today?', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None)]

In [2]:
import chromadb
import uuid
import time

In [ ]:
client = chromadb.PersistentClient(path="db_prac_1")

collection = client.get_or_create_collection(
    name="",
    metadata={"hnsw": {"space": "cosine"}
)

In [ ]:
def create_memory(content: str):
    
    memory_id = str(uuid.uuid4())
    collection.add(
        documents = [content],
        ids = [memory_id]
    )